In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [2]:
#%matplotlib inline

In [3]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHGData1.txt', 'SHGData2.txt', 'SHGData4.txt', 'SHGData5.txt', 'SHGData6.txt', 'SHGData7.txt', 'SHGData8.txt', 'SHGData9.txt', 'SHGData10.txt', 'SHGData11.txt', 'SHGData12.txt', 'SHGData13.txt', 'SHGData14.txt', 'SHGData15.txt', 'SHGData16.txt', 'SHGData17.txt', 'SHGData18.txt', 'SHGData19.txt', 'SHGData20.txt', 'SHGData21.txt', 'SHGData22.txt', 'SHGData23.txt', 'SHGData24.txt', 'SHGData25.txt', 'SHGData26.txt', 'SHGData27.txt', 'SHGData28.txt', 'SHGData29.txt', 'SHGData30.txt', 'SHGData31.txt', 'SHGData32.txt', 'SHGData33.txt', 'SHGData34.txt', 'SHGData35.txt', 'SHGData37.txt', 'SHGData38.txt', 'SHGData39.txt', 'SHGData40.txt', 'SHGData41.txt', 'SHGData42.txt', 'SHGData43.txt', 'SHGData44.txt', 'SHGData45.txt', 'SHGData46.txt', 'SHGData47.txt', 'SHGData48.txt', 'SHGData49.txt', 'SHGData50.txt', 'SHGData3.txt', 'SHGData36.txt']


In [4]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [5]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)
    #print(df)
remove=['SHGData4.txt','SHGData10.txt','SHGData9.txt','SHGData14.txt','SHGData31.txt','SHGData32.txt',\
'SHGData37.txt','SHGData38.txt','SHGData39','SHGData40.txt','SHGData41.txt','SHGData42.txt','SHGData43.txt',\
'SHGData46.txt','SHGData47.txt','SHGData48.txt','SHGData49.txt']
frame = frame.loc[~frame.file.isin(remove)] # Removes data values from these select files. 
#frame = frame.pivot(columns = 'file', index = 'timeDelay', values = 'PDvalue')
#print(frame)
#print(frame[3][0]) # prints out all the data in third column with values corresponding to row 0. 
#print(frame[3][0]) # prints out all the time values in the 1st column and can now interpolate them into time delay. 
# frame2 = frame[[1,3,'date','file']]
# frame[1] = frame[0].str.split('\t')
# print(frame[1])
# print(frame[frame['file']=='SHGData1.txt'])
# frame2.rename(columns={1:'timeDelay',3:'PDsignal'},inplace = True)
# timeDelay = frame2['timeDelay']
# PDvalue = frame2['PDsignal']
# plt.plot(timeDelay[:][:],PDvalue[:][:],'r.')
# plt.show()

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [6]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)
#This is a function that converts the x-axis of the data into stage delay

In [7]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,900):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(900,len(allFiles)-16)
PDvalues = PDvalues.values.reshape(900,len(allFiles)-16) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay

#PDvalues = np.array(PDvalues.tolist()).reshape((900,len(allFiles)-16))
#PDvalues = PDvalues.astype(np.float)
#print(np.where(PDvalues=='-0.-0220')) # array 5,32 Used to find where this funky element was
#AvgPD50 = PDvalues.mean(axis = 1) 

PDvalues[5][-2] = -0.0220 # An error popped up because this looked like: -'0.-220'
PDvalues = PDvalues.astype(np.float) #Ensures all the values are floats so that I can plot them!
AvgPD50 = PDvalues.mean(axis = 1) #average of all the datasets
#timeDelay = ConvertToTimeDelay(times, itr, offset)

In [8]:
extraticks = [-10,-9,-8,-7,-6,-5,-4, -3, -2, -1, 1, 2, 3, 4,5,6,7,8,9,10]
plt.plot(timeDelay,AvgPD50,'b')
plt.xlabel('Stage Delay (ps)')
plt.ylabel('PD Signal (arb)')
plt.grid(True)
plt.xticks(list(plt.xticks()[0]) + extraticks)
plt.xlim(-11,11)
plt.show()
plt.savefig('PD_SignalVsDelay_5_30_2017.png')

In [9]:
?pd.read_csv